In [1]:
# prompt: import the file https://docs.google.com/spreadsheets/d/1o7bLfOYgDyA1Xy73QdqMwdpxLrgO6SN4zQ4VnyMJ500/edit?usp=drive_link from gdrive

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1o7bLfOYgDyA1Xy73QdqMwdpxLrgO6SN4zQ4VnyMJ500/edit?usp=drive_link')

# To read a specific worksheet, replace 'Sheet1' with the actual sheet name
worksheet = spreadsheet.worksheet('Captured Texts')

# Get all of the values from the worksheet as a list of lists
rows = worksheet.get_all_values()

# Print the data (optional)
# for row in rows:
#   print(row)

# You can convert it to a pandas DataFrame if needed
import pandas as pd
df = pd.DataFrame(rows[1:], columns=rows[0])
df

,Date (GMT),Job Link,Origin URL,Name,Average_score,Number_of_reviewers,Taste_profile,Variety,Aroma,Taste,...,Volume,Alcohol_content,Full_price,Action_price,Contains_colouring,Product description,Distillery,Distillery_status,Distillery_founding_year,Distillery_country_region
0,2025-06-15 11:22:56,https://dashboard.browse.ai/workspaces/levente...,https://www.whisky.de/shop/en/product/Jura-Red...,Jura Red Wine Cask Finish,3.8,70,Taste profile\n Fruit\n Sweet\n Sherry,Single Malt Whisky,"Sultanas, creamy caramel and black forest cake...",Red wine with a hint of raspberries and strawb...,...,"0,7 l ·",40 % vol,,"29,90 €",Contains colouring: Caramel color ·,This Jura was first matured in former bourbon ...,The Distillery: Isle of Jura,Open,1810,"Country, Region\nScotland, Islands"
1,2025-06-15 18:04:42,https://dashboard.browse.ai/workspaces/levente...,https://www.whisky.de/shop/en/product/Aberlour...,Aberlour 48% Non Chill-Filtered 12 Years,4.3,309,Taste profile\n Sherry\n Fruit\n Sweet,Single Malt Whisky,"Full and fruity, distantly reminiscent of stra...",Strong sherry notes and ripe fruit followed by...,...,"0,7 l ·",48 % vol,,"21,90 €",Contains colouring: Caramel color ·,The Aberlour Non Chill-Filtered differs from t...,The Distillery: Aberlour,Open,1826,"Country, Region\nScotland, Speyside"
2,2025-06-16 08:59:54,https://dashboard.browse.ai/workspaces/levente...,https://www.whisky.de/shop/en/product/Jura-Red...,Jura Red Wine Cask Finish,3.8,70,Taste profile\n Fruit\n Sweet\n Sherry,Single Malt Whisky,"Sultanas, creamy caramel and black forest cake...",Red wine with a hint of raspberries and strawb...,...,"0,7 l ·",40 % vol,,"29,90 €",Contains colouring: Caramel color ·,This Jura was first matured in former bourbon ...,The Distillery: Isle of Jura,Open,1810,"Country, Region\nScotland, Islands"
3,2025-06-16 09:10:17,https://dashboard.browse.ai/workspaces/levente...,https://www.whisky.de/shop/en/product/Lagavuli...,Lagavulin 16 Years,4.5,1607,Taste profile\n Peat Smoke\n Seaweed\n Maritim...,Single Malt Whisky,Intense peat smoke with notes of iodine and se...,Dry peat smoke fills the palate with a gentle ...,...,"0,7 l ·",43 % vol,,"49,90 €",Contains colouring: Caramel color ·,Lagavulin's classic is kilned with a high smok...,The Distillery: Lagavulin,Open,1816,"Country, Region\nScotland, Islay"
4,2025-06-16 09:10:17,https://dashboard.browse.ai/workspaces/levente...,https://www.whisky.de/shop/en/product/Glenfarc...,Glenfarclas 25 Years,4.6,582,Taste profile\n Sherry\n Oak\n Sweet,Single Malt Whisky,"Jam, honey and a little coffee melt into sherr...",Sherry and oak have the upper hand. Wonderfull...,...,"0,7 l ·",43 % vol,,"198,00 €","282,86 €/l",This Glenfarclas matures for a quarter of a ce...,The Distillery: Glenfarclas,Open,1836,"Country, Region\nScotland, Speyside"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004,2025-06-16 09:32:06,https://dashboard.browse.ai/workspaces/levente...,https://www.whisky.de/shop/en/product/Bowmore-...,Bowmore - new design 15 Years,,,Taste profile\n Peat Smoke\n Sweet\n Sherry,Single Malt Whisky,"Dark chocolate, sun-dried fruit and a hint of ...",Marvellous cedar wood and strong caramel syrup.,...,"0,7 l ·",43 % vol,,"86,90 €",Contains colouring: Caramel color ·,"After maturing in bourbon casks, the extremely...",The Distillery: Bowmore,Open,1779,"Country, Region\nScotland, Islay"
2005,2025-06-16 09:32:06,https://dashboard.browse.ai/workspaces/levente...,https://www.whisky.de/shop/en/product/Rothaus-...,Rothaus Black Forest Banyuls,,,Taste profile\n Nutmeg\n Fruit\n Caramel\n Sherry,Single malt whisky,Warm woody notes and dark dried fruit combine ...,"Fruity, sweet with strong wine notes, plus gen...",...,"0,5 l ·","55,9 % vol",,"59,90 €","119,80 €/l",Rothaus Black Forest Banyuls Finish spent its ...,The Distillery: Rothaus,Open,1791,"Country, Region\nGermany"
2006,2025-06-16 09:32:08,https://dashboard.browse.ai/workspaces/levente...,https://www.whisky.de/s

In [2]:
df.drop(columns=["Region"], inplace=True)

In [3]:
# prompt: replace , to . in the column Action_price, if more than one . are in there, keep only the last,  replace € mark to nothing, cast to float

df['Action_price'] = df['Action_price'].str.replace('€', '', regex=False)
df['Action_price'] = df['Action_price'].str.replace('.', '', regex=False)
df['Action_price'] = df['Action_price'].str.replace(',', '.', regex=False)
df['Action_price'] = pd.to_numeric(df['Action_price'])


In [4]:
# prompt: if for a single row the column Contains_colouring has the € sign in it, copy it's content to the column Full_price, and delete it from the column Contains_colouring

for index, row in df.iterrows():
    if '€' in row['Contains_colouring']:
        #df.loc[index, 'Full_price'] = row['Contains_colouring']
        df.loc[index, 'Contains_colouring'] = ''



In [5]:
# prompt: if the column Contains_colouring has a non empty string, then make it bool true, if empty, bool false

df['Contains_colouring'] = df['Contains_colouring'].apply(lambda x: bool(x.strip()))


In [6]:
# prompt: drop column Full_price

df = df.drop('Full_price', axis=1)
df = df.drop('Job Link', axis=1)
df = df.drop('Date (GMT)', axis=1)



In [7]:
# prompt: Average_score should be casted to float, if empty string, then to 0

df['Average_score'] = df['Average_score'].apply(lambda x: float(x) if x else 0.0)
df['Number_of_reviewers'] = df['Number_of_reviewers'].apply(lambda x: float(x) if x else 0.0)


In [8]:
df["Taste_profile"] = df["Taste_profile"].str.replace("Taste profile\n", "")

In [9]:
df["Taste_list"] = (
    df["Taste_profile"]
    .str.split(r"\s*\\n\s*")          # split on the literal “\n”
    .apply(lambda lst: [t.strip() for t in lst if t.strip()])
)
df["Taste_sentence"] = df["Taste_list"].apply(", ".join)


Original idea: https://chatgpt.com/share/684fef18-8560-8001-a94b-f1ca4d21fa0e

In [10]:
# prompt: df["Taste_profile"] drop column

df = df.drop("Taste_profile", axis=1)

In [11]:
df.Variety.value_counts()


,count
Variety,
Single Malt Whisky,1161
Single malt whisky,255
Kentucky Straight Bourbon,94
Single Malt Whiskey,67
Blended Whisky,58
Blended Malt Whisky,50
Blended whisky,46
Blended Whiskey,44
Rye,30


In [12]:
# prompt: unify the strings of the column Variety to all smallcaps

df['Variety'] = df['Variety'].str.lower()

In [13]:
# prompt: replace all whisky and whiskey in Variety to none and strip the string

df['Variety'] = df['Variety'].str.replace('whisky', '', regex=False)
df['Variety'] = df['Variety'].str.replace('whiskey', '', regex=False)
df['Variety'] = df['Variety'].str.strip()
#df['Variety'] = df['Variety'].str.replace('', 'unknown', regex=False)


In [14]:
# prompt: replace empty string  or single space in Variety to unknown

df['Variety'] = df['Variety'].apply(lambda x: 'unknown' if x.strip() == '' else x)

In [15]:
# prompt: drop rows if column Variety has rum, gin, distillate or liqueur in it, even if any of these has no space after it

df = df[~df['Variety'].str.contains('rum|gin|distillate|liqueur')]

In [16]:
# prompt: replace % vol in Alcohol_content, cast it to float and rename the colun to Alcohol_content_percent

df['Alcohol_content_percent'] = df['Alcohol_content'].str.replace('% vol', '', regex=False)
df['Alcohol_content_percent'] = df['Alcohol_content_percent'].str.replace(',', '.', regex=False)

df['Alcohol_content_percent'] = df['Alcohol_content_percent'].astype(float)
df = df.drop('Alcohol_content', axis=1)

<ipython-input-16-3572085328>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Alcohol_content_percent'] = df['Alcohol_content'].str.replace('% vol', '', regex=False)
<ipython-input-16-3572085328>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Alcohol_content_percent'] = df['Alcohol_content_percent'].str.replace(',', '.', regex=False)
<ipython-input-16-3572085328>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [17]:
# prompt: replace Please select to unknown in the column Distillery_status as well as Torn down to Closed, empty string to unknown

df['Distillery_status'] = df['Distillery_status'].replace({
    'Please select': 'unknown',
    'Torn down': 'Closed',
    '': 'Closed'  # Replacing empty strings
})


In [18]:
# prompt: replace Products with similar taste	in Distillery to unknown, the string The Distillery: to nothing in all rows, and ig empty string remains, to unknown

df['Distillery'] = df['Distillery'].str.replace('Products with similar taste', 'unknown', regex=False)
df['Distillery'] = df['Distillery'].str.replace('The Distillery:', '', regex=False)
df['Distillery'] = df['Distillery'].apply(lambda x: 'unknown' if x.strip() == '' else x)

In [19]:
# prompt: replace Distillery_founding_year empty to nan and cast to integer

import numpy as np

df['Distillery_founding_year'] = df['Distillery_founding_year'].replace('', np.nan)
df['Distillery_founding_year'] = df['Distillery_founding_year'].astype('Int64')

In [20]:
# prompt: replace Country, Region\n in Distillery_country_region to  ''

df["Distillery_country_region"] = df["Distillery_country_region"].str.replace("Country, Region\n", "", regex=False)

In [21]:
# prompt: split Distillery_country_region on , make a new column Country from the first part stripped and a Column Region from the second, stripped, if second is missing, leave the region empty, drop original column

df[['Country', 'Region']] = df['Distillery_country_region'].str.split(',', expand=True)
df['Country'] = df['Country'].str.strip()
df['Region'] = df['Region'].str.strip()
df['Region'] = df['Region'].fillna('') # Fill NaN with empty string for cases with no region
df = df.drop('Distillery_country_region', axis=1)

In [22]:
df.Region.value_counts()

,count
Region,
,698
Speyside,423
Highlands,321
Islay,191
Kentucky,113
Islands,109
Lowlands,66
Campbeltown,26
Tennessee,22


In [23]:
# prompt: split on first space the column Volume, then replace , to . and cast to float

df['Volume'] = df['Volume'].str.split(' ', n=1, expand=True)[0]
df['Volume'] = df['Volume'].str.replace(',', '.', regex=False)
df['Volume'] = df['Volume'].astype(float)

In [24]:
df.Volume.value_counts()

,count
Volume,
0.70,1827
1.00,71
0.50,52
0.20,18
2.10,7
0.05,7
0.75,5
2.80,2
0.60,2


 'Aroma', 'Taste',
       'Finish', 'Maturation', 'Product description'


MAybe region is fucked up into the Taste?

In [25]:
df.Maturation

,Maturation
0,Ex-bourbon casks and finish in red wine casks
1,
2,Ex-bourbon casks and finish in red wine casks
3,Bourbon and sherry casks
4,Oloroso sherry casks
...,...
2004,Bourbon casks and finish in Oloroso sherry casks
2005,
2006,"Hogsheads, finish in first fill Murça port cas..."
2007,Finish in ex Islay casks


In [26]:
df['Product description']

,Product description
0,This Jura was first matured in former bourbon ...
1,The Aberlour Non Chill-Filtered differs from t...
2,This Jura was first matured in former bourbon ...
3,Lagavulin's classic is kilned with a high smok...
4,This Glenfarclas matures for a quarter of a ce...
...,...
2004,"After maturing in bourbon casks, the extremely..."
2005,Rothaus Black Forest Banyuls Finish spent its ...
2006,This 13-year-old North British was bottled by ...
2007,This Invergordon Single Grain was bottled by M...


In [27]:
# prompt: save the df back to Google drive under the name "Whisky.de details - cleaned" as csv. if exists, overwrite.

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to save the CSV file in your Drive
file_path = '/content/drive/My Drive/Whisky.de details - cleaned.csv'

# Save the DataFrame to a CSV file, overwriting if it exists
df.to_csv(file_path, index=False, encoding='utf-8')

print(f"DataFrame saved to {file_path}")


Mounted at /content/drive
DataFrame saved to /content/drive/My Drive/Whisky.de details - cleaned.csv
